In [26]:
import torch
from PoetryDataset import word2ix, ix2word, PoetryDataset
from config import Config
from Model.LSTM import Model

poetry_dataset = PoetryDataset()
cfg = Config()

In [28]:
# 加载模型
# from Model.pretrained_LSTM import Model
model = Model()
model.load_state_dict(torch.load('LSTM_one_direction.pth'))
# model.load_state_dict(torch.load('BiLSTM.pth'))
model.cuda()
model.eval()

RuntimeError: Error(s) in loading state_dict for Model:
	Missing key(s) in state_dict: "lstm.weight_ih_l2", "lstm.weight_hh_l2", "lstm.bias_ih_l2", "lstm.bias_hh_l2", "output.3.weight", "output.3.bias", "output.6.weight", "output.6.bias". 
	Unexpected key(s) in state_dict: "output.2.weight", "output.2.bias", "output.4.weight", "output.4.bias". 

In [22]:
def write_poetry_from_first_sentence(first_sentence):
    poetry = list(first_sentence)
    # 初始隐状态
    bidirectional_num = 2 if cfg.bidirectional else 1
    hidden = torch.zeros((2, cfg.lstm_layers*1, 1, cfg.hidden_size), dtype=torch.float).cuda()
    input = torch.Tensor([word2ix['<START>']]).view(1,1).long().cuda()
    with torch.no_grad():
        for i in range(cfg.seq_len):
            output, hidden = model(input, hidden)
            word_predicted = ''
            # 读取第一句
            if i < len(first_sentence):
                word_predicted = first_sentence[i]
                input = input.data.new([word2ix[word_predicted]]).view(1,1)
            else:
                top_idx = output.data[0].topk(1)[1][0].item()
                word_predicted = ix2word[top_idx]
                poetry.append(word_predicted)
                input = input.data.new([top_idx]).view(1,1)
            
            if word_predicted == '<START>':
                continue
            if word_predicted == '<EOP>' or word_predicted == '<EOP>':
                poetry.pop(-1)
                if len(poetry) < 23:
                    continue
                break
    return ''.join(poetry)
    

def write_poetry_from_first_chars(firstchars):
    poetry = []
    count = 0
    bidirectional_num = 2 if cfg.bidirectional else 1
    hidden = torch.zeros((2, cfg.lstm_layers*1, 1, cfg.hidden_size), dtype = torch.float).cuda()
    input = torch.Tensor([word2ix['<START>']]).view(1,1).long().cuda()
    with torch.no_grad():
        for i in range(cfg.seq_len):
            output, hidden = model(input, hidden)
            word_predicted = ''
            if i == 0 or (poetry[-1] in ['，', '。'] and count < len(firstchars)):
                word_predicted = firstchars[count]
                count += 1
                input = input.data.new([word2ix[word_predicted]]).view(1,1)
            else:
                top_idx = output.data[0].topk(1)[1][0].item()
                word_predicted = ix2word[top_idx]
                input = input.data.new([top_idx]).view(1, 1)
            
            if word_predicted == '<START>':
                continue
            poetry.append(word_predicted)
            if word_predicted == '<EOP>' or word_predicted == '<EOP>':
                poetry.pop(-1)
                if len(poetry) < 23:
                    continue
                break
    return ''.join(poetry)
            

In [24]:
print(write_poetry_from_first_sentence('人闲桂花落'))
print(write_poetry_from_first_sentence('驿外断桥边'))
print(write_poetry_from_first_sentence('赵客缦胡缨'))
print(write_poetry_from_first_sentence('明月几时有'))
print(write_poetry_from_first_sentence('春江潮水连海平'))
print(write_poetry_from_first_sentence('天门中断楚江开'))
print(write_poetry_from_first_sentence('昨夜星辰昨夜风'))
print(write_poetry_from_first_sentence('朝辞白帝彩云间'))
print(write_poetry_from_first_sentence('葡萄美酒夜光杯'))
print(write_poetry_from_first_sentence('东风夜放花千树'))
print(write_poetry_from_first_sentence('紫泉宫殿锁烟霞'))
print(write_poetry_from_first_sentence('花近高楼伤客心'))
print(write_poetry_from_first_sentence('积雨空林烟火迟'))
print(write_poetry_from_first_sentence('身向云山那畔行'))
print(write_poetry_from_first_sentence('落魄江湖载酒行'))

人闲桂花落，春色上阶长。<START>春风满长路，落日满长安。
驿外断桥边，愁人两断肠。不言人不见，独自戍来流。
赵客缦胡缨，秦人传异才。一声南北笛，一曲落中杯。
明月几时有，白云千里归。相思不相见，相见复相依。
春江潮水连海平，白云飞者吹子飞。白鸥白鹭双翅垂，参差褭浪红绡飞。吴儿汉女弄环珮，越女红儿弄浪鬟。舞裙红粉红粉裙，花花笑语红妆红。
天门中断楚江开，一片春愁一片烟。今日送君心似烛，故乡无处不相依。
昨夜星辰昨夜风，一声歌尽更迟迟。若教得似湘妃泪，不肯敎离一夜歌。
朝辞白帝彩云间，曾是人间旧业人。一片烟霞归不得，几家烟火不堪归。江边野店寒侵店，山下孤烟夜对衣。独恨故园无处所，空闻寒雨滴啼痕。
葡萄美酒夜光杯，醉后歌声莫慢狂。莫遣花前醉，醒看日暮催。
东风夜放花千树，落日风吹一片烟。莫怪相思不相见，明朝重作別离来。
紫泉宫殿锁烟霞，洞里仙居几处闻。仙洞仙来无处所，洞庭秋月夜凉风。
花近高楼伤客心，空弹古木有清音。明妃旧手应无点，楚女青蛾不胜羞。
积雨空林烟火迟，故人相见正裴回。风吹野草秋无力，风引寒梅雨半飞。独向高楼思一曲，遥知別路几回催。
身向云山那畔行，几回归去白云飞。几回重到青云路，唯有松风入夜归。
落魄江湖载酒行，一杯一醆欲同归。青云不是人间事，白发还同镜里身。惆怅旧游还似梦，故来无计得相违。


In [25]:
print(write_poetry_from_first_chars('朱健行'))
print(write_poetry_from_first_chars('戢启瑞'))
print(write_poetry_from_first_chars('球王牛逼'))
print(write_poetry_from_first_chars('阎熠光'))
print(write_poetry_from_first_chars('祝捷'))
print(write_poetry_from_first_chars('祝平'))
print(write_poetry_from_first_chars('胡小泉'))
print(write_poetry_from_first_chars('胡昌泰'))
print(write_poetry_from_first_chars('胡晓泉'))
print(write_poetry_from_first_chars('深度学习'))
print(write_poetry_from_first_chars('中国科学院大学'))
print(write_poetry_from_first_chars('我爱自然语言处理'))

朱绂何曾重，健愚自不如。行藏数年事，梦想一年时。
戢，启命复何为。瑞雪飞飞起，春风送羽仪。凤凰传圣旨，飞雨想仙舟。
球。王孙不敢下，牛羊出马前。逼身不得主，出门不敢言。一朝不得意，万里无艰过。大士不得意，高堂空叹息。人生若不昨夜月明天，离人怨
阎巷儿爷。熠狼红杏，光耀黄金。金屋圆光，玉树琼枝。
祝融峰下，捷信天台顶上看。一片白云飞不住，几回青鸟上青天。
祝融峰下，平地天涯是古原。万里烟尘无一处，一川烟草是离人。
胡儿出没燕支山，小犬黄雀走。泉边一个鱼，夜有一个月。
胡儿出没燕支山，昌时城下春云飞。泰山之禹日不得，汉家名臣死亦迟。汉家天子重忠孝，不知何处无功臣。
胡儿出没燕支山，晓织锦丝催断云。泉声不入山头浅，水色遥分落处清。
深夜月明前后时，度花飞絮满庭枝。学君不识人间事，习语应须客语多。
中有一人，国有一匹。科名不改，学行不极。院门寂前垂白露，大是天涯。学得真人，真如不识。
我有一双白鹭之沈梧。爱之不知处，自有青山之上天。然有片石之作天，语而不见心如何。言语不语意不异，处人独见人间有。理性无即是禅，
